In [0]:
!pip install gpyopt

In [0]:
from GPyOpt.methods import BayesianOptimization
import numpy as np

In [0]:
#imports:
import pandas as pd
import numpy as np
import math
import os
import json 

import xgboost
from sklearn.metrics import r2_score

In [4]:
"""
from GPyOpt.methods import BayesianOptimization
import numpy as np

# --- Define your problem
gpyopt_output = []
def f(x): 
  gpyopt_output.append(
        1
    )
  return (6*x-2)**2*np.sin(12*x-4)
domain = [{'name': 'var_1', 'type': 'continuous', 'domain': (0,1)}]

# --- Solve your problem

myBopt = BayesianOptimization(f=f, domain=domain, 
                                 optimize_restarts = 3, initial_design_numdata = 10)
myBopt.run_optimization(max_iter=15)
myBopt.plot_acquisition()
"""

"\nfrom GPyOpt.methods import BayesianOptimization\nimport numpy as np\n\n# --- Define your problem\ngpyopt_output = []\ndef f(x): \n  gpyopt_output.append(\n        1\n    )\n  return (6*x-2)**2*np.sin(12*x-4)\ndomain = [{'name': 'var_1', 'type': 'continuous', 'domain': (0,1)}]\n\n# --- Solve your problem\n\nmyBopt = BayesianOptimization(f=f, domain=domain, \n                                 optimize_restarts = 3, initial_design_numdata = 10)\nmyBopt.run_optimization(max_iter=15)\nmyBopt.plot_acquisition()\n"

In [0]:
#sum(gpyopt_output)

In [6]:
from google.colab import drive 
drive.mount('/mntDrive') 


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /mntDrive


In [0]:
PREP_DATA_PATH = "/mntDrive/My Drive/НИР/data/prepared_facebook_data.csv"
data=pd.read_csv(PREP_DATA_PATH)

RES_SAVE_DIR = "/mntDrive/My Drive/НИР/data/results/test 3"

In [8]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
# get some data
X, y = data.iloc[:,:-1].values, data.iloc[:,-1].values

In [0]:
dtrain = xgboost.DMatrix(X, label=y)

In [0]:
def my_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

def bayes_opt_tune_xgb(para):
  para = para[0]
  params = {
      "tree_method": "gpu_hist", 
      "gpu_id": 0, 
      "verbosity": 0,
      "max_depth": int(round(para[0])),
      "min_child_weight": int(round(para[1])),
      "alpha": para[2],
      "lambda": para[3],
      "subsample": para[4],
      "colsample_bytree": para[5]
      }
    #Cross validating with the specified parameters in 5 folds
  cv_result = xgboost.cv(params, dtrain, nfold=5, feval=my_r2_score,  maximize=True)
  gpyopt_output.append(
        [
         cv_result['test-r2-mean'].iloc[-1],
         params["max_depth"], params["min_child_weight"], 
         params["alpha"], params["lambda"], 
         params["subsample"], params["colsample_bytree"]
        ]
    )
  return cv_result['test-r2-mean'].iloc[-1]

In [0]:
gpyopt_output = []
gpy_importance = pd.DataFrame()
bds = [ {'name': 'max_depth', 'type': 'discrete', 'domain': (3, 10)},
        {'name': 'min_child_weight', 'type': 'discrete', 'domain': (1, 10)},
        {'name': 'alpha', 'type': 'continuous', 'domain': (0, 1)},
        {'name': 'lambda', 'type': 'continuous', 'domain': (0, 1)},
        {'name': 'subsample', 'type': 'continuous', 'domain': (0.5, 0.9)},
        {'name': 'colsample_bytree', 'type': 'continuous', 'domain': (0.5, 0.9)}
      ]
optimizer = BayesianOptimization(f=bayes_opt_tune_xgb, 
                                 domain=bds,
                                 model_type='TPE',
                                 optimize_restarts = 72,
                                 initial_design_numdata = 10,
                                 acquisition_type ='EI',
                                 acquisition_jitter = 0.05,
                                 exact_feval=True, 
                                 maximize=True)

# optimizer.run_optimization(max_iter=719)
optimizer.run_optimization(max_iter=3)

In [0]:
results = pd.DataFrame(gpyopt_output)

In [0]:
results.max(axis = 1)

In [0]:
results.max(axis = 0)